In [194]:
import numpy as np

# Part A

In [210]:
def query(votes, guess):
    return np.sum(np.logical_xor(votes, guess))

In [211]:
members = 20

for i in range(1000):
    
    votes = np.random.randint(low=0, high=2, size=members)
    partition = np.random.randint(low=0, high=members + 1)

    actual_no_votes_partition = partition - np.sum(votes[:partition])
    
    ones = np.ones((members,))
    first = query(votes, ones)

    ones[:partition] = 0
    second = query(votes, ones)

    guess_no_votes_partition = (partition - (second - first)) // 2
    
    assert guess_no_votes_partition == actual_no_votes_partition

# Part B

In [220]:
def find_no_voter(votes, i, j):
    
    num_no_voters = query(votes[i:j], np.ones(j - i))
    
    # If there are no no-voters in this subsequence
    if num_no_voters == 0:
        return None
    
    # If the subsequence only has a single person...
    if (j - i) == 1:
        
        # ... and that person votes no
        if num_no_voters == 1:
            return i
    
        # ... and that person votes yes
        if num_no_voters == 0:
            return None
    
    m = (i + j) // 2
    
    left_num_no_voters = query(votes[i:m], np.ones(m - i))
    right_num_no_voters = query(votes[m:j], np.ones(j - m))
    
    if left_num_no_voters > right_num_no_voters:
        return find_no_voter(votes, i, m)
    else:
        return find_no_voter(votes, m, j)    

In [264]:
members = 8

for i in range(1000):
        
    votes = np.random.randint(low=0, high=2, size=members)
    
    no_voter = find_no_voter(votes, 0, len(votes))
    
    if no_voter is not None:
        assert votes[no_voter] == 0
    else:
        assert members - np.sum(votes) == 0

# Part C